# Практика
Используемые библиотеки

In [4]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [3]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [81]:
# Ваше решение
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
df = df.assign(isNorm = None)
lens = len(df.index)
for index in range(lens):
    normativ = df["Норматив"][index]
    
    result = df["Результат анализа"][index]
    if result == 'б/цвета':
        res = 0
    else:
        res = float(result)
    
    if "в пределах" in normativ:
        val_from = float(re.findall(" (\d*)-", normativ)[0])
        val_to = float(re.findall("-(\d*)", normativ)[0])
        if res >= val_from and res<=val_to:
            df['isNorm'][index] = True
        else:
            df['isNorm'][index] = False
            
    elif "не более" in normativ and "-" in normativ:
        val_from = float(re.findall(" (\d*)-", normativ)[0])
        val_to = float(re.findall("-(\d*)", normativ)[0])
        if res <= val_from and res<=val_to:
            df['isNorm'][index] = True
        else:
            df['isNorm'][index] = False

    elif "не более" in normativ and "-" not in normativ:
        val_from = float(re.findall("не более (.*)", normativ)[0].replace(",", "."))
        if res <= val_from:
            df['isNorm'][index] = True
        else:
            df['isNorm'][index] = False
df.reset_index(drop=True, inplace=True)
df.set_index('Показатель', inplace=True)
df.index
df

,Единица измерений,Результат анализа,Норматив,isNorm
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,True
Запах,баллы,1,не более 2-3,True
Цветность,градусы,б/цвета,не более 30,True
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
Нитраты (по NO3),мг/дм3,24,не более 45,True
Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
Хлориды (Cl),мг/дм3,200,не более 350,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [ ]:
# Вероятность первого апельсина 5/(5+4 - пять апельсинов из девяти
# Второго 4/8   - осталось четыре апельсина из восьми
# Третьего 3/7 - осталось три апельсина из семи

# Всех трех 
# P=(5*4*3)/(7*8*9)=5/42 = 0.119

In [143]:
# Ваше решение
# три раза выбирается фрукт
B1 = ["apple"]*4 + ['orange']*5
B2 = ["apple"]*4 + ['orange']*4
B3 = ["apple"]*4 + ['orange']*3
# задаем число экспериментов
n = 100000
# формируем выборку
df = pd.DataFrame({"vals":list(map(lambda a: [
                           random.choice(B1),
                           random.choice(B2),
                           random.choice(B3)], range(n)
                              ))
                 })
# подсчитываем кол-во совпадений со всеми апельсинами
cnt = 0
for i in range(df.shape[0]):
    if df.vals[i] == ['orange', 'orange', 'orange']:
        cnt = cnt + 1
# находим вероятность        
P_orange = cnt / df.shape[0]
P_orange

0.11942

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [140]:
# Ваше решение

In [ ]:
# вероятность события А1 равна = 3/10,  
# кроме того, P = 7/9 , так как перед взятием второй детали у мастера осталось 9 деталей, 
# из которых только 2 нестандартные и 7 стандартных. По теореме умножения
# P = 3/10 * 7/9 = 7/30

In [157]:
B1 = ["standart"]*7 + ['not_standart']*3
B2 = ["standart"]*7 + ['not_standart']*2
# задаем число экспериментов
n = 100000
# формируем выборку
df = pd.DataFrame({"vals":list(map(lambda a: [
                           random.choice(B1),
                           random.choice(B2)], range(n)
                              ))
                 })
# подсчитываем кол-во совпадедений по деталям
cnt = 0
for i in range(df.shape[0]):
    if df.vals[i] == ['not_standart', 'standart'] :
        cnt = cnt + 1

# находим вероятность        
P_standart = cnt / df.shape[0]
P_standart

0.23293